In [3]:
import json
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv
from IPython.display import display
from ipywidgets import widgets
from pathlib import Path

load_dotenv()

DATA_DIRECTORY = Path(os.environ["DATA_DIRECTORY"])
print(f"{DATA_DIRECTORY=}")

DATA_DIRECTORY=PosixPath('/home/jasper/Projects/code-weekend-1/data')


In [11]:
def plot_test(test_id: int) -> None:
    file = DATA_DIRECTORY / f"{test_id:03d}.json"
    data = json.loads(file.read_text(encoding="utf-8"))

    print(f"Test: {test_id}")

    for stat in ["speed", "power", "range"]:
        print(f"Hero {stat}: {data["hero"][f"base_{stat}"]} ({data["hero"][f"level_{stat}_coeff"]})")

    print(f"Map size: {data["width"]} x {data["height"]}")
    print(f"Start: ({data["start_x"]}, {data["start_y"]})")
    print(f"#turns: {data["num_turns"]}")
    print(f"#monsters: {len(data["monsters"])}")

    df_monsters = pd.DataFrame(data["monsters"])
    df_monsters["y"] = data["height"] - df_monsters["y"]
    df_monsters["gold_per_hp"] = df_monsters["gold"] / df_monsters["hp"]
    df_monsters["exp_per_hp"] = df_monsters["exp"] / df_monsters["hp"]

    figs = []
    for stat in ["gold", "gold_per_hp", "exp", "exp_per_hp", "hp", "attack", "range"]:
        if stat not in df_monsters.columns:
            continue

        fig = px.scatter(df_monsters, x="x", y="y", color=stat, height=600)

        fig.update_layout(xaxis_range=[0, data["width"]])
        fig.update_layout(yaxis_range=[0, data["height"]])
        fig.update_yaxes(scaleanchor="x", scaleratio=1)

        fig.update_layout(showlegend=False)
        fig.add_shape(type="rect", x0=0, y0=0, x1=data["width"], y1=data["height"])

        fig.add_scatter(x=[data["start_x"]], y=[data["height"] - data["start_y"]], marker={"color": "black"})
        fig.add_shape(type="circle",
                      xref="x", yref="y",
                      x0=data["start_x"] - data["hero"]["base_speed"], y0=data["height"] - (data["start_y"] - data["hero"]["base_speed"]),
                      x1=data["start_x"] + data["hero"]["base_speed"], y1=data["height"] - (data["start_y"] + data["hero"]["base_speed"]),
                      line_color="blue")
        fig.add_shape(type="circle",
                      xref="x", yref="y",
                      x0=data["start_x"] - data["hero"]["base_range"], y0=data["height"] - (data["start_y"] - data["hero"]["base_range"]),
                      x1=data["start_x"] + data["hero"]["base_range"], y1=data["height"] - (data["start_y"] + data["hero"]["base_range"]),
                      line_color="red")

        figs.append(go.FigureWidget(fig))

    display(widgets.VBox(figs))

plot_test(45)

Test: 45
Hero speed: 40 (25)
Hero power: 15 (30)
Hero range: 30 (25)
Map size: 2000 x 500
Start: (776, 198)
#turns: 2000
#monsters: 1027


    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<br>gold=%{marker.color}<extra><…